# Analysis of `alarm_violations.csv` for Minute Volume

For each parameter ...
* Concerning alarm violations ...
  * Create a boxplot and stripplot based on the VALUENUM
  * Clean the data if necessary
  * Create histogram for cleaned VALUENUM  
* Concerning alarm thresholds ...
  * Create a boxplot and stripplot based on the THRESHOLD_VALUE (stratified by THRESHOLD_TYPE)
  * Clean the data if necessary
  * Create histogram for cleaned THRESHOLD_VALUE (stratified by THRESHOLD_TYPE)


## Import Data

In [ ]:
# Overview of libraries used
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
alarm_violations = pd.read_csv('./alarm_violations.csv')
alarm_violations.head()

## Parameter: Minute Volume (Volume of Breathing Air Inhaled and Exhaled per Minute)

* `224687` **Minute Volume** (Minute Volume), metavision, in L/min (numeric)
* `220293` **Minute Volume Alarm - High** (MV Alarm - High), metavision, in L/min (numeric)
* `220292` **Minute Volume Alarm - Low** (MV Alarm - Low), metavision, in L/min (numeric)

### Minute Volume - Alarm Violations

In [ ]:
MV_violations = alarm_violations[(alarm_violations['ITEMID'] == 224687)]
display(MV_violations)

In [ ]:
MV_violations.VALUENUM.describe() # values from 0 to 18,918.4

In [ ]:
MV_violations.boxplot(column='VALUENUM')

Among the VALUENUM values is a maximum minute volume of 18,918.4 L/min.
This appears to be at least one implausible outlier.

Let's check whether the unit (VALUEUOM) is "L/min" for all minute volumes (what is expected according to `D_ITEMS.csv`).

In [ ]:
MV_violations.VALUEUOM.unique()

All minute volumes are given in L/min as expected.

Let's check the literature to see what minute volumes can be expected based on medical knowledge.

* Calculation: MV = RR * Tidal Volume
    * Tidal Volume: volume of air moved into or out of the lungs during a normal breath (average volume per breath)
* Average of adults:
    * 13 insp/min * 0.6 L = 7.8 L/min (Hollmann, Wildor. Spiroergometrie: kardiopulmonale Leistungsdiagnostik des Gesunden und Kranken; mit 15 Tabellen. Schattauer Verlag, 2006.)
    * 17 insp/min * 0.5 L = 8.5 L/min (https://www.tu-braunschweig.de/index.php?eID=dumpFile&t=f&f=38200&token=393c8215e4092c382e1d6d057e0d3cd339b59443)
* can be increased:
    * to the three- to fourfold during physical exertion -> 34 L/min (Schwimming 43 L/min)
    * due to pathological conditions (Fever, pulmonary embolism, hyperacidity of the blood or disorders of the respiratory center)
    * by a factor of 15 from trained athletes -> 127.5 L/min, but patients on ICUs don't exercise that heavily
* can be decreased:
    * when asleep
    * when taking sedatives, in people with chronically elevated levels of carbon dioxide, or when the respiratory center is damaged

Decision for now: Consider minute volumes over 50 L/min as implausible values for ICU patients to be removed before further analysis. Assume also a threshold range up to 50 L/min.

In [ ]:
MV_violations_removed = MV_violations[MV_violations['VALUENUM'] > 50]
MV_violations_removed.VALUENUM.describe()

In [ ]:
sns.stripplot(data=MV_violations_removed, x='VALUENUM') # 4 outliers in range 6,321 to 1,819.4

In [ ]:
MV_violations_removed_without_outliers = MV_violations[(MV_violations['VALUENUM'] > 50) & (MV_violations['VALUENUM'] < 6000)]
MV_violations_removed_without_outliers.VALUENUM.describe() # range of 52 to 935

In [ ]:
sns.stripplot(data=MV_violations_removed_without_outliers, x='VALUENUM')

### Minute Volume - Cleaned Alarm Violations

In [ ]:
MV_violations_cleaned = MV_violations[MV_violations['VALUENUM'] <= 50]
MV_violations_cleaned.VALUENUM.describe()

In [ ]:
sns.stripplot(data=MV_violations_cleaned, x='VALUENUM')

### Minute Volume - Alarm Thresholds

Tbc.

### Minute Volume - Cleaned Alarm Thresholds

Tbc.